In [17]:
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 24 21:26:07 2018

@author: Kevin
"""

import pandas as pd
import numpy as np
import time
import pulp  

###讀檔###
Operation_sequence=pd.read_excel("FJSPOperation_sequence.xlsx")
Processing_time=pd.read_excel("FJSPProcessing_time.xlsx")

ops = Operation_sequence.values
pt = Processing_time.values


    

###定義問題###
model = pulp.LpProblem("MIN_makespan", pulp.LpMinimize)

###決策變數###
Cmax = pulp.LpVariable('Cmax',lowBound = 0, cat='Continuous')

Ci = pulp.LpVariable.dicts("start_time",
                           ((i) for i in range(1,6)),
                           lowBound=0,
                           cat='Continuous')
#i=job,j=operation,k=machine
st = pulp.LpVariable.dicts("start_time",
                           ((i, j, k) for i in range(1,6) for j in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0] for k in list(Processing_time.loc[(Processing_time['Operation']==j),'machine'])),
                           lowBound=0,
                           cat='Continuous')

ct = pulp.LpVariable.dicts("completed_time",
                           ((i, j, k) for i in range(1,6) for j in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0] for k in list(Processing_time.loc[(Processing_time['Operation']==j),'machine'])),
                           lowBound=0,
                           cat='Continuous')

x = pulp.LpVariable.dicts("X_binary_var",
                          ((i, j, k) for i in range(1,6) for j in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0] for k in list(Processing_time.loc[(Processing_time['Operation']==j),'machine'])),
                          lowBound=0,
                          cat='Binary')

y = pulp.LpVariable.dicts("Y_binary_var",
                          ((i, j, l, m,k) for i in range(1,6) for j in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0]  for l in range(1,6) for m in [ops[l-1][a] for a in range(1,5) if ops[l-1][a]!=0]  for k in list(set(list(Processing_time.loc[(Processing_time['Operation']==j),'machine']))&set(list(Processing_time.loc[(Processing_time['Operation']==m),'machine']))) if i < l ),
                          lowBound=0,
                          cat='Binary')
#T = pulp.LpVariable.dicts("completed_time",
#                          ((i, j, k) for i in range(1,6) for j in range(1,7) for k in range(1,6)),
#                          lowBound=0,
#                          cat='Continuous')



###目標式###

model += Cmax

###限制式###
for i in range(1,6):
    for j in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0]:
        ma_list=list(Processing_time.loc[(Processing_time['Operation']==j),'machine'])
        model += pulp.lpSum(x[i,j,ma] for ma in ma_list)==1
  
for i in range(1,6):
    for j in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0]:
        ma_list=list(Processing_time.loc[(Processing_time['Operation']==j),'machine'])
        for k in ma_list:
            ptime=int(Processing_time.loc[(Processing_time['Operation']==j)&(Processing_time['machine']==k),['J%d'%i]].iloc[0])
            I = i
            J = j
            K = k
            model += st[I,J,K] + ct[I,J,K] <=x[I,J,K]*1000
            model += ct[I,J,K] >= st[I,J,K] +ptime-(1-x[I,J,K])*1000


for i in range(1,6):
    for l in range(1,6):
        if i<l:
            for o1 in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0]:
                ma1_list=list(Processing_time.loc[(Processing_time['Operation']==o1)&(Processing_time.loc[:]['J%d'%i]!=0),'machine'])
                for o2 in [ops[l-1][o] for o in range(1,5) if ops[l-1][o]!=0]:
                    ma2_list=list(Processing_time.loc[(Processing_time['Operation']==o2)&(Processing_time.loc[:]['J%d'%l]!=0),'machine'])
                    mach_set=list(set(ma1_list)&set(ma2_list))
                    if len(mach_set)>0:
                        for k in mach_set:
                            model += st[i,o1,k]>=ct[l,o2,k]-y[i,o1,l,o2,k]*1000
                            model += st[l,o2,k]>=ct[i,o1,k]-(1-y[i,o1,l,o2,k])*1000

for i in range(1,6):
    op_num=[ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0]
    for j in range(1,len(op_num)):
        preop_num=op_num[j-1]
        curop_num=op_num[j]
        prema_list=list(Processing_time.loc[(Processing_time['Operation']==preop_num)&(Processing_time.loc[:]['J%d'%i]!=0),'machine'])
        curma_list=list(Processing_time.loc[(Processing_time['Operation']==curop_num)&(Processing_time.loc[:]['J%d'%i]!=0),'machine'])
        model += pulp.lpSum(st[i,curop_num,ma1] for ma1 in curma_list) >= pulp.lpSum(ct[i,preop_num,ma2] for ma2 in prema_list)


for i in range(1,6):
    op_list=[ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0]
    last_op=op_list[-1]
    model +=Ci[i]>=pulp.lpSum(ct[i,last_op,k] for k in list(Processing_time.loc[(Processing_time['Operation']==last_op),'machine']))
    model += Cmax >= Ci[i]


start_time = time.time()

model.solve()

pulp.LpStatus[model.status]     

for var in ct:
    var_value = ct[var].varValue
    print( var[0],var[1],var[2] ,var_value)

total_cost = pulp.value(model.objective)
print ('min cost:',total_cost)
print('the elapsed time:%s'% (time.time() - start_time))





1 5 2 18.0
1 5 3 0.0
1 4 4 0.0
1 4 2 148.0
1 2 2 188.0
1 2 3 0.0
1 3 5 193.0
1 3 4 0.0
2 1 3 68.0
2 1 5 0.0
2 4 4 155.0
2 4 2 0.0
2 2 2 0.0
2 2 3 176.0
3 3 5 18.0
3 3 4 0.0
3 5 2 74.0
3 5 3 0.0
3 1 3 0.0
3 1 5 148.0
3 4 4 186.0
3 4 2 0.0
4 3 5 58.0
4 3 4 0.0
4 2 2 0.0
4 2 3 81.0
4 5 2 0.0
4 5 3 161.0
4 4 4 172.0
4 4 2 0.0
5 4 4 64.0
5 4 2 0.0
5 6 2 0.0
5 6 1 152.0
min cost: 193.0
the elapsed time:69.3766884803772


In [4]:
ct[1,5,2].varValue
# for i in st:
#     print(st[i].varValue)
#     print(st[1,5,2].varValue)
#     print( i[0],i[1],i[2])

18.0

In [5]:
for i in x:
    print(i[0],i[1],i[2],'----',x[i].varValue)


1 5 2 ---- 1.0
1 5 3 ---- 0.0
1 4 4 ---- 0.0
1 4 2 ---- 1.0
1 2 2 ---- 1.0
1 2 3 ---- 0.0
1 3 5 ---- 1.0
1 3 4 ---- 0.0
2 1 3 ---- 1.0
2 1 5 ---- 0.0
2 4 4 ---- 1.0
2 4 2 ---- 0.0
2 2 2 ---- 0.0
2 2 3 ---- 1.0
3 3 5 ---- 1.0
3 3 4 ---- 0.0
3 5 2 ---- 1.0
3 5 3 ---- 0.0
3 1 3 ---- 0.0
3 1 5 ---- 1.0
3 4 4 ---- 1.0
3 4 2 ---- 0.0
4 3 5 ---- 1.0
4 3 4 ---- 0.0
4 2 2 ---- 0.0
4 2 3 ---- 1.0
4 5 2 ---- 0.0
4 5 3 ---- 1.0
4 4 4 ---- 1.0
4 4 2 ---- 0.0
5 4 4 ---- 1.0
5 4 2 ---- 0.0
5 6 2 ---- 0.0
5 6 1 ---- 1.0


In [18]:
import plotly
plotly.tools.set_credentials_file(username='swlkevin', api_key='3lvc4tLz4WFTuPUknM39')

In [15]:
'''--------plot gantt chart-------'''
import pandas as pd
import plotly.plotly as py
import plotly.figure_factory as ff
import datetime
from operator import itemgetter


m_keys=[j+1 for j in range(6)]
j_keys=[j for j in range(5)]
index_record=[]
j_record={}

for var in x:
    if x[var].varValue==1:
        start_time=str(datetime.timedelta(seconds=st[var[0],var[1],var[2]].varValue)) # convert seconds to hours, minutes and seconds
        end_time=str(datetime.timedelta(seconds=ct[var[0],var[1],var[2]].varValue))
        j_record[(var[0],var[1],var[2])]=[start_time,end_time]
        index_record.append([var[0],var[1],var[2]])
index_record.sort(key=itemgetter(2))
        

df=[]
for i in index_record:
    df.append(dict(Task='Machine %s'%(i[2]), Start='2018-11-26 %s'%(str(j_record[(i[0],i[1],i[2])][0])), Finish='2018-11-26 %s'%(str(j_record[(i[0],i[1],i[2])][1])),Resource='Job %s'%(i[0])))
    
fig = ff.create_gantt(df, index_col='Resource', show_colorbar=True, group_tasks=True, showgrid_x=True, title='Flexible_Job shop Schedule')
py.iplot(fig, filename='flexible_job_shop_scheduling', world_readable=True)

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~swlkevin/0 or inside your plot.ly account where it is named 'flexible_job_shop_scheduling'


In [ ]:
index_record